# <font color="Teal">Kerasによる深層学習処理入門</font>

- version 0.1 (2019年)
    - 初版（教材作成：一関高専　小池 敦）
- version 0.2 (2022/1/8)
    - 2版（教材作成：一関高専　小池 敦）

本教材では，TensorFlowという深層学習プラットフォームを使用して深層学習の基本的な手順を学びます．  
TensorFlowのハイレベルAPI（TensorFlowを簡単に使えるようにしたもの）であるKerasを使用します．

# 1. ✏️ <font color="Teal">準備</font>


## 1-1. <font color="Teal">教材の複製</font>

まず，本教材を自分のGoogleドライブに保存します．  
以下の手順を行ってください．


1.   <font color="OrangeRed">Chromeブラウザを利用</font>して，本教材にアクセスする．
2.   「ファイル」→<font color="OrangeRed">「ドライブにコピーを保存」</font>を選択し，自分のGoogleドライブに本教材の複製を作る．


教材中のコードを実行する際は，コードの左側にある実行ボタンを押します．


<font color="RoyalBlue">【実習】$3+5=8$ であることを確認する</font>

```
#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#
3 + 5
#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#
```

In [ ]:
3 + 5

## 1-2. <font color="Teal">GPU利用</font>

本教材では計算時間を短くするためにGPUを利用します．
以下の手順を行ってください．


1.   Colabの「ランタイム」→<font color="OrangeRed">「ランタイムのタイプを変更」</font>で，ハードウェアアクセラレータを「None」から<font color="OrangeRed">「GPU」</font>に変更し保存する．  
（既に「GPU」になっている場合はそのままでよい．）


<font color="RoyalBlue">【実習】GPUが割り当てられていることを確認する</font>

```
#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#
!nvidia-smi
#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#
```

In [ ]:
!nvidia-smi

二重線の下の行に，GPUの名前（Tesra K80, Tesla T4 等）が表示されます．  
"NVIDIA-SMI has failed ・・・" と出力される場合は，上記手順が正しく行えていませんので，再度上記手順を行ってください．

# 2. ✏️ <font color="Teal">手書き文字の分類</font>

本章では，[MNIST](http://yann.lecun.com/exdb/mnist/)と呼ばれる手書き数字画像データセットを利用して，手書き数字画像の分類をします．  

## 2-1. <font color="Teal">基本的な使い方（Sequentialモデル）</font> 

* Kerasで深層学習を行う際の流れは以下の通り
    0. データ読み込み，データに対する前処理
    1. モデル（ニューラルネットワークの形）を定義した後，コンパイル（[compile](https://www.tensorflow.org/api_docs/python/tf/keras/Model#compile)）
    2. 学習（[fit](https://www.tensorflow.org/api_docs/python/tf/keras/Model#fit)）
    3. 予測（[predict](https://www.tensorflow.org/api_docs/python/tf/keras/Model#predict)），性能評価（[evaluate](https://www.tensorflow.org/api_docs/python/tf/keras/Model#evaluate)）
* Kerasでのモデル構築方法は基本的に3通り
    * 空のモデルに対し，入力方向から順に層を追加していく（[Sequentialモデル](https://www.tensorflow.org/guide/keras/sequential_model?hl=ja)）←この章の大部分はこれ
    * 自分で自由にネットワークを作る（[Functional API](https://www.tensorflow.org/guide/keras/functional?hl=ja)）←この章の最後に紹介
    * 学習方法なども自分で制御する（[サブクラス化](https://www.tensorflow.org/guide/keras/custom_layers_and_models?hl=ja)）

### 2-1-1. <font color="Teal">TensorFlowの読み込み</font> 

まずはNumPy，matplotlibとTensorFlowを読み込みます．

<font color="RoyalBlue">【実習】TensorFlowをインポートして，tf という別名を付けたのち，バージョンを確認する．NumPy，matplotlibも同時にインポートする．</font>

```
#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
print("TF version:", tf.__version__)
#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#
```

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
print("TF version:", tf.__version__)

### 2-1-2. <font color="Teal">データ読み込み</font> 

MNISTという手書き数字（0〜9）画像のデータを使用します．
* 画像の仕様（[MNIST Webページ](http://yann.lecun.com/exdb/mnist/)より）
    * 28 * 28 ピクセルのモノクロ画像
    * 各ピクセルには0(白)〜255(黒)の値が格納されている
* ラベルの仕様
    * 0から9の10個の数字

<font color="RoyalBlue">【実習】KerasからMNISTデータを読み込む
* あらかじめ学習用データとテスト用データに分けられている
* 詳細は[Kerasドキュメント](https://keras.io/ja/datasets/#mnist)参照
</font>

```
#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#
```

In [ ]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

<font color="RoyalBlue">【実習】読み込んだデータの型をチェックする</font>

```
#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#
print(type(x_train), type(y_train), type(x_test), type(y_test))
#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#
```

In [ ]:
print(type(x_train), type(y_train), type(x_test), type(y_test))

<font color="RoyalBlue">【実習】読み込んだデータのサイズをチェックする
* ndarrayのサイズをチェックするためには[`shape`プロパティ](https://docs.scipy.org/doc/numpy/reference/generated/numpy.ndarray.shape.html)を呼び出す
</font>

```
#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#
print(x_train.shape, y_train.shape, x_test.shape, y_test.shape)
#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#
```

In [ ]:
print(x_train.shape, y_train.shape, x_test.shape, y_test.shape)

<font color="RoyalBlue">【実習】ndarrayに格納されている要素のデータ型をチェックする
* x_trainとy_trainの先頭要素の型を表示する（uint8型は8bitの整数型）
</font>

```
#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#
print(type(x_train[0,0,0]), type(y_train[0]))
#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#
```

In [ ]:
print(type(x_train[0,0,0]), type(y_train[0]))

<font color="RoyalBlue">【実習】読み込んだデータのうち，最初のサンプルの画像を表示する
   * matplotlibの[imshow()関数](https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.imshow.html)を使う
   * `cmap="binary"`はピクセルの色の付け方の指定．詳細は[Choosing Colormaps in Matplotlib](https://matplotlib.org/stable/tutorials/colors/colormaps.html)を参照のこと
   * 各ピクセルの値を確認するため画像の隣にカラーバーを表示する．そのために[colorbar()関数](https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.colorbar.html)を使用する
</font>

```
#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#
plt.imshow(x_train[0], cmap = "binary")
plt.colorbar()
plt.show()
#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#
```

In [ ]:
plt.imshow(x_train[0], cmap = "binary")
plt.colorbar()
plt.show()

<font color="RoyalBlue">【実習】読み込んだデータのうち，最初のサンプルのラベル（答え）を確認する</font>

```
#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#
y_train[0]
#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#
```

In [ ]:
y_train[0]

<font color="RoyalBlue">【実習】先頭30サンプルの画像とラベルを表示する
* [plt.subplot](https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.subplot.html)は複数画像の同時表示を行うためのもの．
5行6列の領域に対して1枚ずつ画像を描いていく（indexは1始まりであることに注意）
* [plt.text](https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.text.html)は指定位置へのテキスト表示．
ここでは画像の右下にサンプルのラベル（答え）を表示するようにする
* [plt.xticks](https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.xticks.html)，[plt.yticks](https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.yticks.html)は軸の表示の指定．何も表示しないようにする
</font>

```
#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#
images = 5 * 6
for i in range(images):
    plt.subplot(5, 6, i + 1)
    plt.imshow(x_train[i], cmap = "binary")
    plt.text(21, 25.5, str(y_train[i]), color="cornflowerblue")
    plt.xticks([])
    plt.yticks([])
plt.show()
#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#
```

In [ ]:
images = 5 * 6
for i in range(images):
    plt.subplot(5, 6, i + 1)
    plt.imshow(x_train[i], cmap = "binary")
    plt.text(21, 25.5, str(y_train[i]), color="cornflowerblue")
    plt.xticks([])
    plt.yticks([])
plt.show()

### 2-1-3. <font color="Teal">画像に対する前処理</font> 

画像の画素値を整数（uint8型）から浮動小数点数に変換し，また，画素値の最大値が1になるようにします．

<font color="RoyalBlue">【実習】各ピクセルが0から1の間の浮動小数点数の値を取るようにする</font>

```
#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#
x_train = x_train / 255
x_test = x_test / 255
print(type(x_train[0,0,0]))
#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#
```

In [ ]:
x_train = x_train / 255
x_test = x_test / 255
print(type(x_train[0,0,0]))

<font color="OrangeRed">(っ´･ω･)っ</font><font color="IndianRed">【演算子 `/` で割り算を行うと，`整数 / 整数` であっても，結果が必ず浮動小数点数になることに注意しましょう．】</font>

### 2-1-4. <font color="Teal">モデルの定義とコンパイル</font> 


[シーケンシャルモデル](https://www.tensorflow.org/guide/keras/sequential_model?hl=ja)を使用して以下の手順で簡単な深層学習モデルを作ります．  
1. 空のモデルを作ったのち [add](https://www.tensorflow.org/api_docs/python/tf/keras/Sequential#add)メソッドにより，入力層から順に層を追加
2. 最後にモデルをコンパイル（[compile](https://www.tensorflow.org/api_docs/python/tf/keras/Sequential#compile)）

コンパイル時には，オプティマイザー，損失関数，メトリックスを指定します．
* オプティマイザー（optimizer）：[損失最小化アルゴリズム](https://www.tensorflow.org/api_docs/python/tf/keras/optimizers)を指定する．どれを選ぶかで性能が変化する．[adam](https://www.tensorflow.org/api_docs/python/tf/keras/optimizers/Adam)などがよく使われる．
* 損失関数（loss）：[損失](https://www.tensorflow.org/api_docs/python/tf/keras/losses)の計算式．扱う問題とデータに応じて変える．
    * 回帰なら，２乗誤差（[mean_squared_error](https://www.tensorflow.org/api_docs/python/tf/keras/losses/MeanSquaredError)）がよく使われる
    * 分類なら，スパースカテゴリカルクロスエントロピー（[sparse_categorical_crossentropy](https://www.tensorflow.org/api_docs/python/tf/keras/losses/SparseCategoricalCrossentropy)）が使われる
    * 分類においてラベルがone-hot表現で与えられる場合は，カテゴリカルクロスエントロピー（[categorical_crossentropy](https://www.tensorflow.org/api_docs/python/tf/keras/losses/CategoricalCrossentropy)）が使われる
* メトリクス（metrics）：モデルの[評価基準](https://www.tensorflow.org/api_docs/python/tf/keras/metrics)．人間がモデルを評価する際に使用する．複数の評価基準を指定可能．
    * 正解率（[accuracy](https://www.tensorflow.org/api_docs/python/tf/keras/metrics/Accuracy)）などが使われる．

<font color="OrangeRed">(っ´･ω･)っ</font><font color="IndianRed">【分類時の損失関数ですが，出力が確率ベクトルで正解ラベルが one-hot 表現の時は，一般的にクロスエントロピー（Kerasの`categorical_crossentropy`）が使われます．一方で正解ラベルが，`0`から`分類数-1`の整数で与えられる場合，ラベルを一旦 one-hot 表現に変換しても良いですが，そうしなくても，損失関数として，Kerasの`sparse_categorical_crossentropy`を使用することで，ラベルを one-hot 表現に変換してからクロスエントロピーを計算する場合と同じ損失を計算してくれるようになります．】</font>

<font color="RoyalBlue">【実習】簡単なモデルを定義する
* 出力は10次元の確率ベクトル（10個数字それぞれに対する確率）とする
* 中間層として全結合層（[Dense](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Dense)）を一つだけ入れる
* 後続の処理（Dense層）のために，モデルの最初に(28, 28) の二次元の画像を728(=28*28)要素の一次元に変換する処理を入れる（[Flatten](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Flatten)）
* 出力層は全結合で softmax を行う
</font>

```
#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Flatten(input_shape=(28, 28)))
model.add(tf.keras.layers.Dense(64, activation="relu"))
model.add(tf.keras.layers.Dense(10, activation="softmax"))
#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#
```

In [ ]:
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Flatten(input_shape=(28, 28)))
model.add(tf.keras.layers.Dense(64, activation="relu"))
model.add(tf.keras.layers.Dense(10, activation="softmax"))

<font color="OrangeRed">(っ´･ω･)っ</font><font color="IndianRed">【ラベルがスカラー（0から9の数字）であるにもかかわらず，このモデルの出力は10次元ベクトルになっています．上述の通り，コンパイル時（後述）の `loss` に `sparse_categorical_crossentropy` を指定すると，ラベルの数字をone-hotベクトル表現に自動変換し，そのone-hotベクトルとモデル出力の誤差（カテゴリカルクロスエントロピー）を計算するようになります．】</font>

<font color="OrangeRed">(っ´･ω･)っ</font><font color="IndianRed">【Sequentialモデルの別の記述方法として，以下のように各層をリストとして列挙することもできます．  
```
model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28)),
    tf.keras.layers.Dense(64, activation="relu"),
    tf.keras.layers.Dense(10, activation="softmax")
])
```
】</font>

<font color="OrangeRed">(っ´･ω･)っ</font><font color="IndianRed">【毎回，`tf.karas.layers.Dense()` のように書くのが面倒な場合は，事前に  
`from tensorflow.keras.layers import Dense`  
と書いておくことで  
`tf.karas.layers.Dense()` → `Dense()`  
と書けるようになります．  
その書き方をすると以下のようになります．  
```
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense

model = Sequential()
model.add(Flatten(input_shape=(28, 28)))
model.add(Dense(64, activation="relu"))
model.add(Dense(10, activation="softmax"))
```
】</font>

<font color="RoyalBlue">【実習】作成したネットワークの概要を確認する</font>

```
#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#
model.summary()
#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#
```

In [ ]:
model.summary()

<font color="RoyalBlue">【実習】モデルをコンパイルする</font>

```
#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#
model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])
#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#
```

In [ ]:
model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])

<font color="OrangeRed">(っ´･ω･)っ</font><font color="IndianRed">【compile メソッドにおいて，optimizer, loss, metricsにはクラスのインスタンスを指定しますが，上記ソースコードのように文字列で指定することもできます．デフォルトの設定以外を使用する場合は，文字列でなくクラスのインスタンスで指定する必要があり，例えば[adam](https://www.tensorflow.org/api_docs/python/tf/keras/optimizers/Adam)を学習率0.01で使用するには，上記`compile`メソッドの中で  
`optimizer=tf.keras.optimizers.Adam(learning_rate=0.01)`  
のように指定します．
】</font>

### 2-1-5. <font color="Teal">モデルの学習</font> 


モデルを学習させるには[fit](https://www.tensorflow.org/api_docs/python/tf/keras/Sequential#fit)メソッドを使用します．その際，以下のパラメータを指定します．
* `epochs`：学習データを何回モデルに入れるかを指定する．`epochs=20` とすると各学習データは20回ずつモデルに入力される．
* `batch_size`：何サンプルごとにモデルのパラメータ更新を行うかを指定する．

<font color="RoyalBlue">【実習】作成したモデルで学習を行う</font>

```
#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#
model.fit(x_train, y_train, epochs=20, batch_size=128)
#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#
```

In [ ]:
model.fit(x_train, y_train, epochs=20, batch_size=128)

### 2-1-6. <font color="Teal">モデルの評価</font> 


<font color="RoyalBlue">【実習】テストデータを使って，作成したモデルの正解率を計算する</font>


```
#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#
test_loss, test_acc = model.evaluate(x_test, y_test)
print('Test accuracy:', test_acc)
#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#
```

In [ ]:
test_loss, test_acc = model.evaluate(x_test, y_test)
print('Test accuracy:', test_acc)

<font color="RoyalBlue">【実習】テストデータの最初の画像を分類する
* `NumPy`の`argmax`関数を使うと，10次元ベクトルから最も確率の高い次元（どの数字が最も確率が高いか？）を計算することができる．
</font>

```
#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#
print(model.predict(x_test[[0]]))
print(np.argmax(model.predict(x_test[[0]])))
#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#
```

In [ ]:
print(model.predict(x_test[[0]]))
print(np.argmax(model.predict(x_test[[0]])))

<font color="OrangeRed">(っ´･ω･)っ</font><font color="IndianRed">【上記の`x_test[[0]]`において，`[[0]]` という風に括弧が2重になっていることに注意が必要です．`predict`メソッドを使って予測をする際，メソッドには複数画像を同時に入力できるようになっており，引数として（画像枚数, 28ピクセル, 28ピクセル）の3次元配列（3階テンソル）を渡すようになっています．引数を `x_test[0]` としてしまうと，（28ピクセル,28ピクセル）の2次元配列（行列）になってしまうため，`x_test[[0]]` とすることで，（画像1枚,28ピクセル,28ピクセル）の3次元配列を渡します．なお，`x_test[[0]]` というのは，`x_test[ ]`にリスト `[0]` を渡す書き方（詳しくはNumPyドキュメントの[Indexing with Arrays of Indices](https://numpy.org/doc/stable/user/quickstart.html#indexing-with-arrays-of-indices)をご覧ください）で，リストで指定したインデックスの要素を取り出すことができます．ここでは，リストとして`[0]`が指定されているため，index 0の画像のみが取り出されます．この代わりに，`x_test[[0],:,:]` や `x_test[0:1]` と書いても同じになります．
】</font>

<font color="RoyalBlue">【実習】テストデータの最初の30枚の画像を分類する</font>

```
#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#
images = 5 * 6
predictions = np.argmax(model.predict(x_test[:images]), axis=1)
for i in range(images):
    plt.subplot(5, 6, i + 1)
    plt.imshow(x_test[i], cmap = "binary")
    plt.text(21, 25.5, str(y_test[i]), color="cornflowerblue")
    plt.text(0, 25.5, str(predictions[i]), color="red")
    if predictions[i] != y_test[i]:
        plt.plot([0, 27], [1, 1], color='red', linewidth=5)
    plt.xticks([])
    plt.yticks([])
#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#
```

In [ ]:
images = 5 * 6
predictions = np.argmax(model.predict(x_test[:images]), axis=1)
for i in range(images):
    plt.subplot(5, 6, i + 1)
    plt.imshow(x_test[i], cmap = "binary")
    plt.text(21, 25.5, str(y_test[i]), color="cornflowerblue")
    plt.text(0, 25.5, str(predictions[i]), color="red")
    if predictions[i] != y_test[i]:
        plt.plot([0, 27], [1, 1], color='red', linewidth=5)
    plt.xticks([])
    plt.yticks([])

<font color="OrangeRed">(っ´･ω･)っ</font><font color="IndianRed">【上記の
```np.argmax(model.predict(x_test[:images]), axis=1)```
は複数枚の画像のそれぞれに対して，予測結果を得るための標準的な書き方ですが，`np.argmax()` において，`axis=1`が指定されていることに注意してください．この指定により，各画像の分類結果対して，個別に argmax （値が最大となるindex）を計算できるようになります．  
`model.predict()` を行うと，10次元の横ベクトル（分類結果の横ベクトル）が画像枚数分生成されます．
[`np.argmax()`](https://numpy.org/doc/stable/reference/generated/numpy.argmax.html) を `axis=1`で呼び出すと，この2次元配列に対して横ベクトルごとに個別に argmax を計算するようになります．`axis` を指定しない場合，全要素を一列に並べたベクトルに対して argmax が計算されることになるので，今回の場合には適しません．】</font>


<font color="RoyalBlue">【実習】間違ったサンプルの最初の30枚を表示する</font>

```
#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#
result = np.argmax(model.predict(x_test), axis=1)
x_fail = x_test[result != y_test]
y_fail = y_test[result != y_test]
result_fail = result[result != y_test]
print("fails:", len(x_fail))
images = 5 * 6
for i in range(images):
    plt.subplot(5, 6, i + 1)
    plt.imshow(x_fail[i].reshape((28, 28)), cmap = "binary")
    plt.text(21, 25.5, str(y_fail[i]), color="cornflowerblue")
    plt.text(0, 25.5, str(result_fail[i]), color="red")
    plt.plot([0, 27], [1, 1], color='red', linewidth=5)
    plt.xticks([])
    plt.yticks([])
#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#
```

In [ ]:
result = np.argmax(model.predict(x_test), axis=1)
x_fail = x_test[result != y_test]
y_fail = y_test[result != y_test]
result_fail = result[result != y_test]
print("fails:", len(x_fail))
images = 5 * 6
for i in range(images):
    plt.subplot(5, 6, i + 1)
    plt.imshow(x_fail[i].reshape((28, 28)), cmap = "binary")
    plt.text(21, 25.5, str(y_fail[i]), color="cornflowerblue")
    plt.text(0, 25.5, str(result_fail[i]), color="red")
    plt.plot([0, 27], [1, 1], color='red', linewidth=5)
    plt.xticks([])
    plt.yticks([])

## 2-2. <font color="Teal">畳み込みニューラルネットワーク</font> 

畳み込み層を使うことで画像のパターン認識に強いネットワークを作ることができます．  
畳み込み層には縦×横×チャネル数 からなる画像が入力されますが，元のMNIST画像はチャネル数に関する次元を持っていないので，ネットワークの先頭で，(28, 28)画像から(28, 28, 1)画像に変換します（Reshape）．  
その他の層は以下の通りです．


*   [Conv2D](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Conv2D)：畳み込み層
*   [MaxPooling2D](https://www.tensorflow.org/api_docs/python/tf/keras/layers/MaxPool2D)：マックスプーリング
*   [Dropout](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Dropout)：ドロップアウト
*   [Flatten](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Flatten)：多次元配列を一次元配列に変換

他には [バッチ正規化](https://www.tensorflow.org/api_docs/python/tf/keras/layers/BatchNormalization) などもよく使われます．


<font color="RoyalBlue">【実習】畳み込み層を活用したネットワークを作成する</font>

```
#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#
cnn = tf.keras.models.Sequential()
cnn.add(tf.keras.layers.Reshape((28, 28, 1), input_shape=(28, 28)))
cnn.add(tf.keras.layers.Conv2D(16, (3, 3), activation='relu'))
cnn.add(tf.keras.layers.Conv2D(32, (3, 3), activation='relu'))
cnn.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))
cnn.add(tf.keras.layers.Conv2D(64, (3, 3), activation='relu'))
cnn.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))
cnn.add(tf.keras.layers.Dropout(0.2))
cnn.add(tf.keras.layers.Flatten())
cnn.add(tf.keras.layers.Dense(128, activation='relu'))
cnn.add(tf.keras.layers.Dropout(0.2))
cnn.add(tf.keras.layers.Dense(10, activation='softmax'))
cnn.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])
#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#
```

In [ ]:
cnn = tf.keras.models.Sequential()
cnn.add(tf.keras.layers.Reshape((28, 28, 1), input_shape=(28, 28)))
cnn.add(tf.keras.layers.Conv2D(16, (3, 3), activation='relu'))
cnn.add(tf.keras.layers.Conv2D(32, (3, 3), activation='relu'))
cnn.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))
cnn.add(tf.keras.layers.Conv2D(64, (3, 3), activation='relu'))
cnn.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))
cnn.add(tf.keras.layers.Dropout(0.2))
cnn.add(tf.keras.layers.Flatten())
cnn.add(tf.keras.layers.Dense(128, activation='relu'))
cnn.add(tf.keras.layers.Dropout(0.2))
cnn.add(tf.keras.layers.Dense(10, activation='softmax'))
cnn.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])

<font color="RoyalBlue">【実習】作成したネットワークの概要を確認する</font>

```
#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#
cnn.summary()
#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#
```

In [ ]:
cnn.summary()

<font color="RoyalBlue">【実習】作成したネットワークで学習を行う</font>

```
#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#
cnn.fit(x_train, y_train, epochs=20, batch_size=128)
#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#
```

In [ ]:
cnn.fit(x_train, y_train, epochs=20, batch_size=128)

<font color="RoyalBlue">【実習】作成したネットワークを評価する</font>

```
#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#
test_loss, test_acc = cnn.evaluate(x_test, y_test)
test_acc
#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#
```

In [ ]:
test_loss, test_acc = cnn.evaluate(x_test, y_test)
test_acc

<font color="RoyalBlue">【実習】作成したネットワークで間違ったもののうち30枚を表示する</font>

```
#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#
result = np.argmax(cnn.predict(x_test), axis=1)
x_fail = x_test[result != y_test]
y_fail = y_test[result != y_test]
result_fail = result[result != y_test]
print("fails:", len(x_fail))
pics = 5 * 6
for i in range(pics):
    plt.subplot(5, 6, i + 1)
    plt.imshow(x_fail[i].reshape((28, 28)), cmap = "binary")
    plt.text(21, 25.5, str(y_fail[i]), color="cornflowerblue")
    plt.text(0, 25.5, str(result_fail[i]), color="red")
    plt.plot([0, 27], [1, 1], color='red', linewidth=5)
    plt.xticks([])
    plt.yticks([])
#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#
```

In [ ]:
result = np.argmax(cnn.predict(x_test), axis=1)
x_fail = x_test[result != y_test]
y_fail = y_test[result != y_test]
result_fail = result[result != y_test]
print("fails:", len(x_fail))
pics = 5 * 6
for i in range(pics):
    plt.subplot(5, 6, i + 1)
    plt.imshow(x_fail[i].reshape((28, 28)), cmap = "binary")
    plt.text(21, 25.5, str(y_fail[i]), color="cornflowerblue")
    plt.text(0, 25.5, str(result_fail[i]), color="red")
    plt.plot([0, 27], [1, 1], color='red', linewidth=5)
    plt.xticks([])
    plt.yticks([])

## 2-3. <font color="Teal">検証データによる性能チェック</font> 

検証データを活用して，学習途中の性能をチェックします．  
これにより過学習が発生しているかどうかを確認することができます．  


<font color="RoyalBlue">【実習】上記で作ったCNNの構造を複製して新しいモデルを作り，コンパイルする</font>

```
#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#
cnn = tf.keras.models.clone_model(cnn)
cnn.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])
#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#
```


In [ ]:
cnn = tf.keras.models.clone_model(cnn)
cnn.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])

<font color="OrangeRed">(っ´･ω･)っ</font><font color="IndianRed">【[`clone_model`](https://www.tensorflow.org/api_docs/python/tf/keras/models/clone_model)メソッドはネットワークの構造を複製して新しいネットワークを作りますが，重みは複製されません．】</font>

<font color="RoyalBlue">【実習】検証データを使って性能の変化をチェックしながら学習を行う
* `validation_split`を指定すると，学習データ中の指定した割合を検証用に使用する
* 学習中の性能の変化は[`fit`](https://www.tensorflow.org/api_docs/python/tf/keras/Model#fit)メソッドの戻り値であるヒストリーオブジェクトに含まれている
</font>

```
#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#
history = cnn.fit(x_train, y_train, epochs=20, batch_size=128, validation_split=0.2)
#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#
```


In [ ]:
history = cnn.fit(x_train, y_train, epochs=20, batch_size=128, validation_split=0.2)

<font color="OrangeRed">(っ´･ω･)っ</font><font color="IndianRed">【今回は極端な過学習は起こっていないですが，このまま検証データを用いた過学習対策の実習を続けます．】</font>


<font color="RoyalBlue">【実習】学習中の性能の変化を描画する</font>

```
#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#
import matplotlib.pyplot as plt

plt.plot(history.history['accuracy'], 'o-', label='Training')
plt.plot(history.history['val_accuracy'], 'o-', label='Validation')
plt.legend()
plt.title('Accuracy')
plt.show()

plt.plot(history.history['loss'], 'o-', label='Training')
plt.plot(history.history['val_loss'], 'o-', label='Validation')
plt.legend()
plt.title('Loss')
plt.show()
#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#
```


In [ ]:
import matplotlib.pyplot as plt

plt.plot(history.history['accuracy'], 'o-', label='Training')
plt.plot(history.history['val_accuracy'], 'o-', label='Validation')
plt.legend()
plt.title('Accuracy')
plt.show()

plt.plot(history.history['loss'], 'o-', label='Training')
plt.plot(history.history['val_loss'], 'o-', label='Validation')
plt.legend()
plt.title('Loss')
plt.show()

<font color="RoyalBlue">【実習】テストデータを使って，作成したモデルの正解率を計算する</font>


```
#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#
test_loss, test_acc = cnn.evaluate(x_test, y_test)
test_acc
#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#
```


In [ ]:
test_loss, test_acc = cnn.evaluate(x_test, y_test)
test_acc

## 2-4. <font color="Teal">早期終了</font>

検証データでの性能向上が見られなくなったら学習を打ち切るようにすることができます．  
[`fit`](https://www.tensorflow.org/api_docs/python/tf/keras/Model#fit)メソッドのコールバックのリスト（`callbacks`）に[`EarlyStopping`](https://www.tensorflow.org/api_docs/python/tf/keras/callbacks/EarlyStopping)を含めると，指定した回数性能向上がない場合に学習を打ち切るようになります．

<font color="RoyalBlue">【実習】上記で作ったCNNの構造を複製して新しいモデルを作り，コンパイルする</font>

```
#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#
cnn = tf.keras.models.clone_model(cnn)
cnn.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])
#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#
```


In [ ]:
cnn = tf.keras.models.clone_model(cnn)
cnn.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])

<font color="RoyalBlue">【実習】検証データを使って性能の変化をチェックしながら学習し，性能が3エポック向上しなかったら学習を終了する
</font>

```
#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#
history = cnn.fit(x_train, y_train, epochs=40, batch_size=128, validation_split=0.2, 
                  callbacks = [tf.keras.callbacks.EarlyStopping(monitor="val_loss", patience=3, verbose=1)])
#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#
```


In [ ]:
history = cnn.fit(x_train, y_train, epochs=40, batch_size=128, validation_split=0.2, 
                  callbacks = [tf.keras.callbacks.EarlyStopping(monitor="val_loss", patience=3, verbose=1)])

<font color="RoyalBlue">【実習】学習中の性能の変化を描画する</font>

```
#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#
import matplotlib.pyplot as plt

plt.plot(history.history['accuracy'], 'o-', label='Training')
plt.plot(history.history['val_accuracy'], 'o-', label='Validation')
plt.legend()
plt.title('Accuracy')
plt.show()

plt.plot(history.history['loss'], 'o-', label='Training')
plt.plot(history.history['val_loss'], 'o-', label='Validation')
plt.legend()
plt.title('Loss')
plt.show()
#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#
```


In [ ]:
import matplotlib.pyplot as plt

plt.plot(history.history['accuracy'], 'o-', label='Training')
plt.plot(history.history['val_accuracy'], 'o-', label='Validation')
plt.legend()
plt.title('Accuracy')
plt.show()

plt.plot(history.history['loss'], 'o-', label='Training')
plt.plot(history.history['val_loss'], 'o-', label='Validation')
plt.legend()
plt.title('Loss')
plt.show()

<font color="RoyalBlue">【実習】テストデータを使って，作成したモデルの正解率を計算する</font>


```
#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#
test_loss, test_acc = cnn.evaluate(x_test, y_test)
test_acc
#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#
```


In [ ]:
test_loss, test_acc = cnn.evaluate(x_test, y_test)
test_acc

## 2-5. <font color="Teal">TensorBoard 利用</font>

TensorBoardというGUIツールを使って，学習中の性能変化を表示させます．

<font color="RoyalBlue">【実習】上記で作ったCNNの構造を複製して新しいモデルを作り，コンパイルする</font>

```
#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#
cnn = tf.keras.models.clone_model(cnn)
cnn.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])
#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#
```


In [ ]:
cnn = tf.keras.models.clone_model(cnn)
cnn.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])

<font color="RoyalBlue">【実習】TensorBoard用のコールバック設定を含めて学習を行う</font>

```
#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#
from datetime import datetime
logs = "logs/" + datetime.now().strftime("%Y%m%d-%H%M%S")

tboard_callback = tf.keras.callbacks.TensorBoard(log_dir = logs,
                                                 histogram_freq = 1)

history = cnn.fit(x_train, y_train, epochs=20, batch_size=128, validation_split=0.2, 
                  callbacks = [tboard_callback, tf.keras.callbacks.EarlyStopping(monitor="val_loss", patience=3, verbose=1)])

#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#
```


In [ ]:
from datetime import datetime
logs = "logs/" + datetime.now().strftime("%Y%m%d-%H%M%S")

tboard_callback = tf.keras.callbacks.TensorBoard(log_dir = logs,
                                                 histogram_freq = 1)

history = cnn.fit(x_train, y_train, epochs=20, batch_size=128, validation_split=0.2, 
                  callbacks = [tboard_callback, tf.keras.callbacks.EarlyStopping(monitor="val_loss", patience=3, verbose=1)])

<font color="RoyalBlue">【実習】TensorBoardを起動し，学習中の性能変化を表示する</font>

```
#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#
%load_ext tensorboard
%tensorboard --logdir=logs
#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#
```


In [ ]:
%load_ext tensorboard
%tensorboard --logdir=logs

## 2-6. <font color="Teal">Functional API</font>

Kerasにおいて [Functional API](https://www.tensorflow.org/guide/keras/functional?hl=ja) を使うと層ごとの接続関係も自分で記述できるようになります．  
ニューラルネットワークの各層は入力を出力に変換する関数とみなすことができますが， Keras の各層も引数を取ることができ，引数を入力すると出力を返します．
これを利用してモデルを定義する方法を Functional API モードと呼んでいます．

このような記法による長所のひとつとして，ふたつの層で別々に行った処理の結果を統合したりすることができるようになります（ここでは詳細は扱いません）．ある層の出力結果 `x1` と別の層の出力結果 `x2` を統合する代表的な方法として，以下のものがあります．
* ふたつの結果を要素ごとに足し合わせる：[tf.keras.layers.Add()([x1, x2])](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Add)
* ふたつの結果を要素ごとに掛け算する：[tf.keras.layers.Multiply()([x1, x2])](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Multiply)
* `x1` の後ろに `x2` を結合させて大きな出力を作る：[tf.keras.layers.Concatenate()([x1, x2])](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Concatenate)

以下では，導入として，Sequentialモデルで作成した簡単なモデルと同じモデルをFunctional API で作成してみます．

<font color="RoyalBlue">【実習】[2-1-4節](#scrollTo=PuLXdLRbLLMH)で作成したネットワークをFunctional APIを使って作成する</font>

```
#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#
inputs = tf.keras.Input(shape=(28, 28))
x = tf.keras.layers.Flatten(input_shape=(28, 28))(inputs)
x = tf.keras.layers.Dense(64, activation="relu")(x)
outputs = tf.keras.layers.Dense(10, activation="softmax")(x)
model = tf.keras.Model(inputs, outputs)
#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#
```


In [ ]:
inputs = tf.keras.Input(shape=(28, 28))
x = tf.keras.layers.Flatten(input_shape=(28, 28))(inputs)
x = tf.keras.layers.Dense(64, activation="relu")(x)
outputs = tf.keras.layers.Dense(10, activation="softmax")(x)
model = tf.keras.Model(inputs, outputs)

<font color="OrangeRed">(っ´･ω･)っ</font><font color="IndianRed">【上記のように Functional API でモデルを定義する際，入力が与えられる訳ではないので，実際の値の計算は行われません．単に計算手順を計算グラフという形式で記憶しているだけです(例えば，`outputs` は具体的な値ではなく，値を計算するための計算手順を保持しています)
．実際の値の計算は`fit()` を呼び出した際に行われます．】</font>


<font color="RoyalBlue">【実習】上記`output`の型を調べる</font>

```
#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#
print(type(outputs))
print(outputs)
#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#
```


In [ ]:
print(type(outputs))
print(outputs)

<font color="RoyalBlue">【実習】作成したネットワークの概要を確認する</font>

```
#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#
model.summary()
#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#
```


In [ ]:
model.summary()

<font color="RoyalBlue">【実習】作成したネットワークの概要を層ごとの接続関係も含めて可視化する</font>

```
#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#
tf.keras.utils.plot_model(model, "first_model.png", show_shapes=True)
#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#
```


In [ ]:
tf.keras.utils.plot_model(model, "first_model.png", show_shapes=True)

<font color="RoyalBlue">【実習】モデルをコンパイルする</font>

```
#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#
model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])
#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#
```


In [ ]:
model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])

<font color="RoyalBlue">【実習】作成したモデルで学習を行う</font>

```
#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#
model.fit(x_train, y_train, epochs=100, batch_size=128, validation_split=0.2, 
          callbacks = [tf.keras.callbacks.EarlyStopping(monitor="val_loss", min_delta=1e-2, patience=3, verbose=1)])
#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#
```


In [ ]:
model.fit(x_train, y_train, epochs=100, batch_size=128, validation_split=0.2, 
          callbacks = [tf.keras.callbacks.EarlyStopping(monitor="val_loss", min_delta=1e-2, patience=3, verbose=1)])

<font color="RoyalBlue">【実習】テストデータを使って，作成したモデルの正解率を計算する</font>


```
#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#
test_loss, test_acc = model.evaluate(x_test, y_test)
test_acc
#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#
```


In [ ]:
test_loss, test_acc = model.evaluate(x_test, y_test)
test_acc

## 2-7. <font color="Teal">演習</font>

* [FASION-MNIST](https://keras.io/ja/datasets/#fashion-mnist)を使用して洋服画像の分類モデルを作る
    * https://keras.io/ja/datasets/#fashion-mnist
    * データ読み込み：`(x_train, y_train), (x_test, y_test) = tf.keras.datasets.fashion_mnist.load_data()`